In [1]:
import numpy as np
import nibabel as nib
import pandas as pd
import os,sys
import math
import os.path
import cv2
import pydicom
from PIL import Image
import cv2 as cv
import random
import pickle
import time
import pydicom
from scipy import ndimage as ndi
#from torch import absolute_eigenvaluesh
from scipy import absolute
from itertools import combinations_with_replacement
from PIL import ImageEnhance
import copy

In [2]:
d3_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0],
             [0,0,1],[-1,-1,1],[0,-1,1],[1,-1,1],[1,0,1],[1,1,1],[0,1,1],[-1,1,1],[-1,0,1],
             [0,0,-1],[-1,-1,-1],[0,-1,-1],[1,-1,-1],[1,0,-1],[1,1,-1],[0,1,-1],[-1,1,-1],[-1,0,-1]]
d2_plat_1 = [[0,0,0],[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0]]
d3_plat_1_5 = [[0,0,0],[-5,-5,0],[0,-5,0],[5,-5,0],[5,0,0],[5,5,0],[0,5,0],[-5,5,0],[-5,0,0],
               [0,0,5],[-5,-5,5],[0,-5,5],[5,-5,5],[5,0,5],[5,5,5],[0,5,5],[-5,5,5],[-5,0,5],
               [0,0,-5],[-5,-5,-5],[0,-5,-5],[5,-5,-5],[5,0,-5],[5,5,-5],[0,5,-5],[-5,5,-5],[-5,0,-5]]
d3_plat_1_no0 = [[-1,-1,0],[0,-1,0],[1,-1,0],[1,0,0],[1,1,0],[0,1,0],[-1,1,0],[-1,0,0],
             [0,0,1],[-1,-1,1],[0,-1,1],[1,-1,1],[1,0,1],[1,1,1],[0,1,1],[-1,1,1],[-1,0,1],
             [0,0,-1],[-1,-1,-1],[0,-1,-1],[1,-1,-1],[1,0,-1],[1,1,-1],[0,1,-1],[-1,1,-1],[-1,0,-1]]
d3_plat_0 = [[0,0,0]]
d2_plat_1_4 = [[0,-1],[1,0],[0,1],[-1,0]]
d2_plat_1_8 = [[-1,-1],[0,-1],[1,-1],[1,0],[1,1],[0,1],[-1,1],[-1,0]]
d3_plat_8 = [[0,0,0],[1,0,0],[1,1,0],[0,1,0],
             [0,0,1],[1,0,1],[1,1,1],[0,1,1]]
d3_plat_8_no0 = [[1,0,0],[1,1,0],[0,1,0],
             [0,0,1],[1,0,1],[1,1,1],[0,1,1]]
six_direction = [[-1,0,0],[1,0,0],[0,-1,0],[0,1,0],[0,0,-1],[0,0,1]]
small_patch = 15
big_patch = 64
patch0 = small_patch
patch1 = small_patch
patch2 = small_patch
patch3 = big_patch
patch4 = big_patch
patch5 = big_patch

In [13]:
windows = 1

In [220]:
file_path1 = r'E:\article4\样例文件\P'
file_number = '100'
coregister = [63, 184, 85, 236, 205, 259]
t1_threshold = 1000
t1_type = 0
xt1,xt2,yt1,yt2,zt1,zt2 = 78,184,103,226,203,243
cita = math.pi*0
xt3,xt4 = 115,140

In [9]:
def read_data(path):#定义一个“读取图像”函数，用来读取图像
    image_data = nib.load(path)
    return image_data

In [10]:
def get_image(file_number):
    if windows == 1:
        file_path = file_path1
        path_p02 = file_path+file_number+'\dp'+file_number+'_mag_insert2_n4.nii'#图像路径，日后可以根据需要修改
    else:
        file_path = file_path0
        path_p02 = file_path+file_number+'/dp'+file_number+'_mag_insert2_n4.nii'#图像路径，日后可以根据需要修改
    p02_img = read_data(path_p02)#使用“读取图像”函数

    p02_3d = p02_img.get_fdata()
    p02_3d.shape
    return p02_3d

In [19]:
def make_groups(origin,g_number,n_number):
    groups = []
    now_points = []
    repeats = []
    g = 0
    number = -1
    for x,y,z in origin:
        number = number+1
        if number%1000 == 0:
            print(number)
        #if g >= 1:
            #break
        
        #length_group = len(repeats)#开始迭代时分组个数
        #print([x,y,z])
        if [x,y,z] not in repeats:#确定不在其他小组中
            if [x,y,z] not in now_points:#确定不会重复统计本小组中的点
                already_points = []

                if len(now_points) == 0:#如果是小组中第一个点，就干脆把周围一圈的点都计算进来
                    now_points.append([x,y,z])
                    for i,j,k in d3_plat_1_no0:
                        if [x+i,y+j,z+k] in origin:
                            now_points.append([x+i,y+j,z+k])
                            already_points.append([x+i,y+j,z+k])
                
                for iteration in range(n_number):
                    length = len(now_points)#开始迭代时当前小组坐标点个数
                    #print(iteration)
                    temporary_points = []

                    for i,j,k in d3_plat_1_no0:#如果不是小组中第一个点，就判断周围是不是有这个小组的点，如果有就加入进来
                        for x1,y1,z1 in already_points:
                            if [x1+i,y1+j,z1+k] in origin:
                                if [x1+i,y1+j,z1+k] not in now_points:
                                    now_points.append([x1+i,y1+j,z1+k])
                                    temporary_points.append([x1+i,y1+j,z1+k])

                    already_points = []
                    for x,y,z in temporary_points:
                        already_points.append([x,y,z])

                    if len(now_points) == length:#如果坐标点个数没有增加就结束迭代 
                        #print(length)
                        groups.append(now_points)#将不再增加的这一小组坐标点作为一个小组加入到分组中
                        for xxx,yyy,zzz in groups[g]:#将这一小组列入已经计算的小组，防止重复计算
                            repeats.append([xxx,yyy,zzz])
                        
                        now_points = []#这一小组坐标点归零，准备开始新的一个小组
                        g = g+1
                        #print(g)
                        break
            
    return groups

In [68]:
path_t1 = file_path1+file_number+'\dp'+file_number+'_t1.nii'
target_t1 = read_data(path_t1)#使用“读取图像”函数

target_3d_t1 = target_t1.get_fdata()
target_3d_t1.shape

(256, 320, 256)

In [227]:
target_same_insert = get_image(file_number)

In [80]:
a,b,c = target_3d_t1.shape
target_t1_region = np.zeros((a,b,c))
for x in range(xt1,xt2):
    for y in range(yt1,yt2):
        for z in range(zt1,zt2):
            if t1_type == 0:
                if target_3d_t1[x,y,z] < t1_threshold:
                    target_t1_region[x,y,z] = 255
            else:
                if target_3d_t1[x,y,z] > t1_threshold:
                    target_t1_region[x,y,z] = 255

In [81]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_t1_region, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_t1_region.nii')

In [82]:
for x in range(96,106):
    for y in range(181,208):
        for z in range(217,224):
            target_t1_region[x,y,z] = 0

In [83]:
for x in range(101,114):
    for y in range(208,215):
        for z in range(213,219):
            target_t1_region[x,y,z] = 0

In [84]:
a,b,c = target_t1_region.shape
csf_sembles = []
for x in range(a):
    for y in range(b):
        for z in range(c):
            if target_t1_region[x,y,z] == 255:
                csf_sembles.append([x,y,z])
print(len(csf_sembles))
csf_groups = make_groups(csf_sembles,10000,1000000)

93375
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000


In [85]:
lenths = []
for n in range(len(csf_groups)):
    lenths.append(len(csf_groups[n]))
for n in range(len(csf_groups)):
    if len(csf_groups[n])==max(lenths):
        print(n,len(csf_groups[n]),csf_groups[n][0])
        choose_number = n
        break

13 75830 [82, 117, 203]


In [86]:
a,b,c = target_t1_region.shape
target_csf_region = np.zeros((a,b,c))
for n in range(len(csf_groups[choose_number])):
    x,y,z = csf_groups[choose_number][n]
    target_csf_region[x,y,z] = 255

In [87]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_csf_region, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_csf_region.nii')

In [92]:
def get_region_surface(ifile,group):
    surface_group = []
    for n in range(len(group)):
        x,y,z = group[n]
        number = 0
        for i,j,k in d3_plat_1_no0:
            #print([x+i,y+j,z+k],number)
            if ifile[x+i,y+j,z+k]==0:
                number += 1
        if number>6:
            surface_group.append([x,y,z])
    return surface_group

In [93]:
t1_surface_group = get_region_surface(target_t1_region,csf_groups[choose_number])

In [94]:
a,b,c = target_3d_t1.shape
target_t1_surface = np.zeros((a,b,c))
for n in range(len(t1_surface_group)):
    x,y,z = t1_surface_group[n]
    target_t1_surface[x,y,z] = 255

In [95]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_t1_surface, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_t1_surface.nii')

In [213]:
def expand_3d(ifile,group,min_d,max_d,cita):
    expand_surface = []
    a,b,c = ifile.shape
    for x in range(xt1-int(max_d+2),xt2+int(max_d+2)):
        print(x)
        for y in range(yt1-int(max_d+2),yt2+int(max_d+2)):
            for z in range(zt1-int(max_d+2),zt2+int(max_d+2)):
                #print([x,y,z])
                if [x,y,z] not in group:
                    #print([x,y,z])
                    near_in = False
                    near_out = False
                    if x<a/2:
                        #near_in = False
                        if ifile[x+int(round(max_d*math.cos(cita))),y+int(round(max_d*math.sin(cita))),z]==255:
                            near_in = True
                        #near_out = False
                        if ifile[x+int(round(min_d*math.cos(cita))),y+int(round(min_d*math.sin(cita))),z]==255:
                            near_out = True
                    else:
                        #near_in = False
                        if ifile[x-int(round(max_d*math.cos(cita))),y-int(round(max_d*math.sin(cita))),z]==255:
                            near_in = True
                        #near_out = False
                        if ifile[x-int(round(min_d*math.cos(cita))),y-int(round(min_d*math.sin(cita))),z]==255:
                            near_out = True
                    #print(near_in,near_out)
                    if near_out == True:
                        pass
                    else:
                        if near_in == True:
#                             distance1 = get_distance_3d(x,y,z,x1,y1,z1)
#                             if distance1<max_d:
#                         min_distance = 10000
#                         for n in range(len(array_group)):
#                             #print(array_group[n])
#                             x1,y1,z1 = array_group[n]
#                             #if (x1-int(a/2))*(x-int(a/2))>0 and (y1-int(b/2))*(y-int(b/2))>0:

#                             distance = get_distance_3d(x,y,z,x1,y1,z1)
#                             if distance<min_distance:
#                                 min_distance = distance
#                         if min_d<=min_distance<=max_d:
                            expand_surface.append([x,y,z])
    return expand_surface

In [214]:
def get_distance_3d(x1,y1,z1,x2,y2,z2):
    distance_3d = math.sqrt((x1-x2)**2+(y1-y2)**2+((z1-z2))**2)
    return distance_3d

In [217]:
begin_time = time.time()
expand_surface_group = expand_3d(target_csf_region,csf_groups[choose_number],7,9,cita)
print(time.time()-begin_time)

67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
1783.6837260723114


In [223]:
a,b,c = target_3d_t1.shape
target_t1_surface_expand = np.zeros((a,b,c))
for n in range(len(expand_surface_group)):
    x,y,z = expand_surface_group[n]
    if x<xt3 or x>xt4:
        target_t1_surface_expand[x,y,z] = 255

In [224]:
affine = np.diag([1, 2, 3,1])
img_3d = nib.Nifti1Image(target_t1_surface_expand, affine)
img_3d.shape
nib.save(img_3d, file_path1+file_number+r'\dp'+file_number+r'_t1_surface_expand.nii')

In [229]:
def bilinear(org_img, org_shape, dst_shape):
    dst_img = np.zeros((dst_shape[0], dst_shape[1], dst_shape[2]))
    dst_h, dst_w, dst_d = dst_shape
    org_h, org_w, org_d = org_shape
    for i in range(dst_h):
        print(i)
        for j in range(dst_w):
            for k in range(dst_d):
                src_x = i * float(org_h / dst_h)
                src_y = j * float(org_w / dst_w)
                src_z = k * float(org_d / dst_d)
                src_x_int = i * org_h // dst_h
                src_y_int = j * org_w // dst_w
                src_z_int = k * org_d // dst_d
                a = src_x - src_x_int
                b = src_y - src_y_int
                c = src_z - src_z_int

                if src_x_int+1 == org_h or src_y_int+1 == org_w or src_z_int+1 == org_d:
                    dst_img[i, j, k] = org_img[src_x_int, src_y_int, src_z_int]
                    continue
                # print(src_x_int, src_y_int)
                dst_img[i, j, k] = (1. - a) * (1. - b) * (1. - c) * org_img[src_x_int+1, src_y_int+1, src_z_int] + \
                                   (1. - a) * b * (1. - c) * org_img[src_x_int, src_y_int+1, src_z_int] + \
                                    a * (1. - b) * (1. - c) * org_img[src_x_int+1, src_y_int, src_z_int] + \
                                    a * b * (1. - c) * org_img[src_x_int, src_y_int, src_z_int] + \
                                    (1. - a) * (1. - b) * c * org_img[src_x_int+1, src_y_int+1, src_z_int] + \
                                   (1. - a) * b * c * org_img[src_x_int, src_y_int+1, src_z_int] + \
                                    a * (1. - b) * c * org_img[src_x_int+1, src_y_int, src_z_int] + \
                                    a * b * c * org_img[src_x_int, src_y_int, src_z_int]
    
    return dst_img

In [276]:
target_t1_surface_expand_crop = target_t1_surface_expand[coregister[0]+18:coregister[1]+10,coregister[2]+10:coregister[3]-10,coregister[4]+4:coregister[5]+4]
img = target_t1_surface_expand_crop
img_shape = (img.shape[0],img.shape[1],img.shape[2])
print(img.shape,target_same_insert.shape)
dst_shape = (target_same_insert.shape[0],target_same_insert.shape[1],target_same_insert.shape[2])
target_t1_surface_expand_crop_insert = bilinear(img, img_shape, dst_shape)
a,b,c = target_t1_surface_expand_crop_insert.shape
target_t1_surface_expand_crop_insert2 = np.zeros((a,b,c))
for x in range(a):
    for y in range(b):
        for z in range(c):
            if target_t1_surface_expand_crop_insert[x,y,z]>=125:
                target_t1_surface_expand_crop_insert[x,y,z]=255
            else:
                target_t1_surface_expand_crop_insert[x,y,z]=0
for x in range(a):
    for y in range(b):
        for z in range(c):
            if target_t1_surface_expand_crop_insert[x,y,z]==255:
                target_t1_surface_expand_crop_insert2[a-1-x,y,z] = 255
affine = np.diag([1, 2, 3,1]) 
img_3d = nib.Nifti1Image(target_t1_surface_expand_crop_insert2, affine) 
img_3d.shape 
nib.save(img_3d,file_path1+file_number+'\dp'+file_number+'_boundary.nii')

(142, 102, 32) (540, 380, 120)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269

In [277]:
def m_filter_exp(ifile,x,y,z,step=3):
    sum_s=[]
    for i in range(-int(step/2),int(step/2)+1):
        for j in  range(-int(step/2),int(step/2)+1):
            for k in  range(-int(step/2),int(step/2)+1):
                sum_s.append(ifile[x+i][y+j][z+k])
    sum_s.sort()
    #print(sum_s)
    return sum_s[int(step*step*step/2)+1]

In [278]:
a,b,c = target_t1_surface_expand_crop_insert2.shape
target_boundary = np.zeros((a,b,c))

for x in range(1,a-1):
    print(x)
    for y in range(1,b-1):
        for z in range(1,c-1):
            if target_t1_surface_expand_crop_insert2[x,y,z] == 0:
                target_boundary[x,y,z] = m_filter_exp(target_t1_surface_expand_crop_insert2,x,y,z)
            else:
                target_boundary[x,y,z] = 255

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [279]:
affine = np.diag([1, 2, 3,1]) 
img_3d = nib.Nifti1Image(target_boundary, affine) 
img_3d.shape 
nib.save(img_3d,file_path1+file_number+'\dp'+file_number+'_boundary1.nii')